### Importar en los notebooks
```python
import sys
sys.path.append('..')
from utils.paths import PATHS, PARAMS, get_file_path, get_data_path


### Acceso directo
```python
df = pd.read_csv(get_file_path('cleaned', 'exportaciones'))
paises = pd.read_csv(get_file_path('mappings', 'paises'))

### O acceso por carpeta
```python
raw_folder = get_data_path('raw')
cleaned_folder = get_data_path('cleaned')

### Construir ruta personalizada
```python
mi_archivo = get_data_path('raw', 'nuevo_archivo.csv')

### Usando `get_file_path`

In [10]:
import pandas as pd
import sys
sys.path.append('..')
from utils.paths import get_file_path


In [11]:
df = pd.read_csv(get_file_path('mappings', 'paises'))

In [12]:
df.head()

,Código,Nombre
0,13,Afganistán
1,15,Islas Åland
2,17,Albania
3,23,Alemania
4,24,Antártida
